In [1]:
import os 
import graphlab as gl

In [2]:
# Read & Save data locally to avoid repeated downloads. 
if os.path.exists('statistics_topics.csv'):
    stats_topics = gl.SFrame.read_csv('data/statistics_topics.csv', header=False)
else:
    stats_topics = gl.SFrame.read_csv('http://s3.amazonaws.com/dato-datasets/tag_lists/statistics_topics.csv', 
                                      header=False)
    stats_topics.save('data/statistics_topics.csv', format='csv')
    

[INFO] This non-commercial license of GraphLab Create is assigned to matthew.krey@flatironschool.com and will expire on December 14, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-98274 - Server binary: /Users/matthewkrey/anaconda2/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1453589500.log
[INFO] GraphLab Server Version: 1.8


PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/tag_lists/statistics_topics.csv to /var/tmp/graphlab-matthewkrey/98274/c3542e71-3588-4ad2-b243-be08d588f3c7.csv
PROGRESS: Unable to parse line "Generalized additive model for location, scale and shape"
PROGRESS: Unable to parse line "Lies, damned lies, and statistics"
PROGRESS: 2 lines failed to parse correctly
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/tag_lists/statistics_topics.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.023869 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Unable to parse line "Generalized additive model for location, scale and shape"
PROGRESS: Unable to parse line "Lies, damned lies,

In [3]:
# See README for details and more info on these stats topics 
stats_topics = stats_topics.rename({'X1': 'tag'})
stats_topics.tail(10)

tag
z statistic
Z-test
Z-transform
Zakai equation
Zelen's design
Zero–one law
Zeta distribution
Ziggurat algorithm
Zipf–Mandelbrot law
Zipf's law


In [4]:
# Create Autotagger model. See README for details on Autotagger
model = gl.autotagger.create(stats_topics)

PROGRESS: Starting brute force nearest neighbors model training.


In [5]:
model.list_fields()

['distance',
 'verbose',
 'num_features',
 'tag_name',
 'training_time',
 'num_examples',
 'features']

In [6]:
# Run to find results of Autotagger model
model.tag?

In [7]:
# Read in document data 
if os.path.exists('stats_overflow_clean'):
    posts = gl.SFrame('stats_overflow_clean')
else:
    posts = gl.SFrame('http://s3.amazonaws.com/dato-datasets/stats_overflow_clean')
    posts.save('data/stats_overflow_clean')

PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stats_overflow_clean/dir_archive.ini to /var/tmp/graphlab-matthewkrey/98274/4fc0cfa4-b263-4ffc-b545-99016a16e817.ini
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stats_overflow_clean/objects.bin to /var/tmp/graphlab-matthewkrey/98274/e4eaa6fd-e863-4989-a714-a23463f36947.bin
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stats_overflow_clean/m_ee1cef21c977f86a.frame_idx to /var/tmp/graphlab-matthewkrey/98274/ee5a8942-1769-4d9c-adeb-691325353707.frame_idx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stats_overflow_clean/m_ee1cef21c977f86a.sidx to /var/tmp/graphlab-matthewkrey/98274/6c29c1b7-6a6b-4c66-be4a-7704fd9e724f.sidx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stats_overflow_clean/m_ee1cef21c977f86a.0000 to /var/tmp/graphlab-matthewkrey/98274/c24f090e-c078-40b6-9f36-02f81b0893a4.0000
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/stats_overflow_clean/m

In [8]:
print "Number of posts:", posts.num_rows()
posts[['Body', 'Title', 'PostTypeId', 'Tags']].tail(5)

Number of posts: 11077


Body,Title,PostTypeId,Tags
"This grew too long for acomment, but I think ...",None,2,None
Assume a classificationproblem where there are ...,Detecting a consistentpattern in a dataset via ...,1,<classification><cross-validation><decision- ...
My goal is to create aformula that can give an ...,How to project videoviewcount based on ...,1,<summary-statistics><median><eviews> ...
As a practical answer tothe real questions yo ...,None,2,None
Decision trees arenotoriously <strong>u ...,None,2,None


In [10]:
posts['doc'] = posts['Title'] + ' ' + posts['Body']

In [11]:
# Query Autotagger Model
tags = model.tag(posts, query_name='doc', k=5, similarity_threshold=0.1)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 692     | 0.00228834  | 35.515ms     |
PROGRESS: | 4153         | 1.1e+07 | 37.4944     | 1.05s        |
PROGRESS: | 8612         | 2.4e+07 | 77.7524     | 2.04s        |
PROGRESS: | Done         |         | 100         | 2.59s        |
PROGRESS: +--------------+---------+-------------+--------------+


In [12]:
# Print query results
tags.print_rows(10, max_row_width=110, max_column_width=40)

+--------+-----------------------------------------+-----------------------------------------+----------------+
| doc_id |                   doc                   |                   tag                   |     score      |
+--------+-----------------------------------------+-----------------------------------------+----------------+
|   13   | neural network output layer for bina... |          Binary classification          | 0.15503875969  |
|   13   | neural network output layer for bina... |        Artificial neural network        | 0.107913669065 |
|   13   | neural network output layer for bina... |         One-class classification        | 0.101449275362 |
|   13   | neural network output layer for bina... |              Neural network             | 0.100775193798 |
|   13   | neural network output layer for bina... |        Multiclass classification        | 0.10071942446  |
|   37   | Negative predictions for binomial pr... |        Negative predictive value        | 0.1047120

In [20]:
# Record linkage of house listings using hypothetical real estate data 
col_types = {'street_number': str, 'postcode': str}
address_features = ['street_number', 'address_1', 'suburb', 'state', 'postcode']

if os.path.exists('febrl_F_org_5000.csv'):
    post_address = gl.SFrame.read_csv('data/febrl_F_org_5000.csv', column_type_hints=col_types)
else:
    url = 'http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_org_5000.csv'
    post_address = gl.SFrame.read_csv(url, column_type_hints=col_types)
    post_address.save('data/febrl_F_org_5000.csv')

PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_org_5000.csv to /var/tmp/graphlab-matthewkrey/98274/e18d271e-733d-43f0-a56f-6115d789de98.csv
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_org_5000.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.034027 secs.
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_org_5000.csv
PROGRESS: Parsing completed. Parsed 3000 lines in 0.016802 secs.


In [21]:
# Print address_features from dataset 
post_address = post_address[address_features]
post_address.print_rows(5)

+---------------+---------------------+---------------+-------+----------+
| street_number |      address_1      |     suburb    | state | postcode |
+---------------+---------------------+---------------+-------+----------+
|       95      |     galway place    |    st marys   |       |   2681   |
|       12      |    burnie street    |   wycheproof  |  nsw  |   2234   |
|       16      | macrobertson street |    branxton   |  qld  |   3073   |
|      170      |    bonrook street   | brighton east |  nsw  |   3087   |
|       32      |   proserpine court  |   helensvale  |  qld  |   2067   |
+---------------+---------------------+---------------+-------+----------+
[3000 rows x 5 columns]



In [22]:
# Create record linker model. See README for details on record linker model
model = gl.record_linker.create(post_address, distance='jaccard')
model.summary()

PROGRESS: Starting brute force nearest neighbors model training.
Class                               : RecordLinker

Schema
------
Number of examples                  : 3000
Number of feature columns           : 5
Number of distance components       : 1
Method                              : brute_force

Training
--------
Total training time (seconds)       : 0.4197



In [23]:
# List model fields
model.list_fields()

['distance',
 'verbose',
 'num_features',
 'num_distance_components',
 'training_time',
 'num_examples',
 'method',
 'features']

In [24]:
# Read query data 
if os.path.exists('febrl_F_dup_5000.csv'):
    agent_listings = gl.SFrame.read_csv('data/febrl_F_dup_5000.csv', column_type_hints=col_types)
else:
    url = 'http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_dup_5000.csv'
    agent_listings = gl.SFrame.read_csv(url, column_type_hints=col_types)
    agent_listings.save('data/febrl_F_dup_5000.csv')

PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_dup_5000.csv to /var/tmp/graphlab-matthewkrey/98274/63cecc12-60f4-453e-9a1d-8b81fc8830a2.csv
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_dup_5000.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.027403 secs.
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/febrl_synthetic/febrl_F_dup_5000.csv
PROGRESS: Parsing completed. Parsed 2000 lines in 0.013885 secs.


In [25]:
# Query the model
model.link?

In [26]:
# Investigate data matches 
matches = model.link(agent_listings, k=None, radius=0.5)
matches.head(5)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 2000    | 0.0333333   | 4.664ms      |
PROGRESS: | Done         |         | 100         | 260.995ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
1,2438,0.41935483871,1
1,533,0.5,2
2,688,0.192307692308,1
3,2947,0.0454545454545,1
5,1705,0.047619047619,1


In [27]:
# Evaluate results 
# The results mean that the address in query row 1 match the address in refs row number 2438.
# Although the Jaccard distance is relatively high at 0.42, manual inspection of data shows poor match.
print agent_listings[1]
print post_address[2438] 

{'phone_number': '08 35547759', 'surname': 'sv', 'street_number': '329', 'age': 27, 'rec_id': 26695, 'suburb': 'smithfield plains', 'state': 'vic', 'date_of_birth': 19305324, 'given_name': 'irne', 'address_2': '', 'blocking_number': 8, 'soc_sec_id': 4201032, 'postcode': '5044', 'address_1': ''}
{'suburb': 'smithfield plains', 'state': 'vic', 'address_1': 'sculptor street', 'street_number': '16', 'postcode': '5044'}


In [28]:
# Evaluate second case based on Jaccard distance 
print agent_listings[3]
print post_address[2947]

{'phone_number': '04 87915906', 'surname': 'teaegzr', 'street_number': '15', 'age': 37, 'rec_id': 13886, 'suburb': 'kallangur', 'state': 'nss', 'date_of_birth': 19459057, 'given_name': 'tristran', 'address_2': '', 'blocking_number': 0, 'soc_sec_id': 6314427, 'postcode': '2506', 'address_1': 'mawalan street'}
{'suburb': 'kallangur', 'state': 'nsw', 'address_1': 'mawalan street', 'street_number': '12', 'postcode': '2506'}


In [30]:
# Data still not a true match. Use Composite Distance function for improved matching.
# See README for details of Composite Distance function, Levenshtein distance & Jaccard distance 
address_dist = [
                 [['street_number'], 'levenshtein', 1],
                 [address_features, 'jaccard', 1]
                ]

model2 = gl.record_linker.create(post_address, distance=address_dist)
model2.summary()

Defaulting to brute force instead of ball tree because there are multiple distance components.
PROGRESS: Starting brute force nearest neighbors model training.
Class                               : RecordLinker

Schema
------
Number of examples                  : 3000
Number of feature columns           : 5
Number of distance components       : 2
Method                              : brute_force

Training
--------
Total training time (seconds)       : 0.3596



In [31]:
# Return model distance 
model2['distance']

[[['street_number'], 'levenshtein', 1],
 [['street_number', 'address_1', 'suburb', 'state', 'postcode'], 'jaccard', 1]]

In [32]:
# Query new model for a large number of neighbors 
pre_match = model2.link(agent_listings, k=10, verbose=False)
pre_match['distance'].show()

Canvas is accessible via web browser at the URL: http://localhost:49844/index.html
Opening Canvas in default web browser.
